In [3]:
import pandas as pd
# Read HotpotQA results HyDE
hotpotqa_df = pd.read_parquet('/Users/chengze/Desktop/hotpotqa_question_answer_type_200.parquet')
print(hotpotqa_df)

                                               question  \
1116  Out to Win is an American documentary film tha...   
1368  Are both Variety and The Advocate LGBT-interes...   
422   Who is this American rapper, songwriter, recor...   
413   In which year was this American country music ...   
451   What is the nationality of the actor who starr...   
...                                                 ...   
163   Kelly Jean Van Dyke is the niece of which Amer...   
909   Which Superbowl featured Jack Maitland and was...   
297   This American comedy-drama film directed by La...   
755   Which city is north east of Normanby Island an...   
405   Which headlined event did Romy Ruyssen face of...   

                              answer        type  
1116                 Houston Rockets      bridge  
1368                              no  comparison  
422                         Lil' Kim      bridge  
413                             2000      bridge  
451                         Scottish

In [17]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

hotpotqa_df = pd.read_parquet('/Users/chengze/Desktop/HyDE_NaiveRAG_hotpotqa_response_results_answer_200.parquet')
client = OpenAI(api_key="")

def get_gpt4_response(answer, response):
    try:
        gpt_response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Please evaluate if the response matches the reference answer."},
                {"role": "user", "content": f"""Instructions
You will receive a ground truth answer (referred to as Answer) and a model-generated answer (referred to as Response). Your task is to compare the two and determine whether they align.

Note: The ground truth answer may sometimes be embedded within the model-generated answer. You need to carefully analyze and discern whether they align.
Your Output:
If the two answers align, respond with yes.
If they do not align, respond with no.
If you are very uncertain, respond with unclear.
Your response should first include yes, no, or unclear, followed by an explanation.

Example 1
Answer: Houston Rockets
Response: The basketball player who was drafted 18th overall in 2001 is Jason Collins, who was selected by the Houston Rockets.
Expected output: yes

Example 2
Answer: no
Response: Yes, both Variety and The Advocate are LGBT-interest magazines. The Advocate is explicitly identified as an American LGBT-interest magazine, while Variety, although primarily known for its coverage of the entertainment industry, also addresses topics relevant to the LGBT community.
Expected output: no

Input Data Format
Ground Truth Answer: {answer}
Model Generated Answer: {response}

Expected Output
yes, no, or unclear
An explanation of your choice.

Output:"""}
            ],
            temperature=0,
            max_tokens=1000
        )
        return gpt_response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error getting GPT-4 response: {e}")
        return str(e)

for idx, row in tqdm(hotpotqa_df.iterrows(), total=len(hotpotqa_df), desc="Processing evaluations"):
    evaluation = get_gpt4_response(row['answer'], row['response'])
    hotpotqa_df.at[idx, 'gpt4_evaluation'] = evaluation

hotpotqa_df.to_parquet('/Users/chengze/Desktop/HyDE_NaiveRAG_hotpotqa_response_results_answer_200_updated.parquet')
print(hotpotqa_df)


Processing evaluations: 100%|██████████| 200/200 [07:57<00:00,  2.39s/it]

                                              question  \
0    Out to Win is an American documentary film tha...   
1    Are both Variety and The Advocate LGBT-interes...   
2    Who is this American rapper, songwriter, recor...   
3    In which year was this American country music ...   
4    What is the nationality of the actor who starr...   
..                                                 ...   
195  Kelly Jean Van Dyke is the niece of which Amer...   
196  Which Superbowl featured Jack Maitland and was...   
197  This American comedy-drama film directed by La...   
198  Which city is north east of Normanby Island an...   
199  Which headlined event did Romy Ruyssen face of...   

                                    retrieval_document  \
0    [(0.9446477, 'Sudanese-born Australian profess...   
1    [(0.817297, 'ts and tourists on the right.  On...   
2    [(0.95737755, 'response to Nicki Minaj\'s song...   
3    [(0.9063984, 'first tropical cyclone to make l...   
4    [(0.8443

In [18]:
# Convert GPT-4 evaluations to binary scores
import pandas as pd
df = pd.read_parquet('/Users/chengze/Desktop/HyDE_NaiveRAG_hotpotqa_response_results_answer_200_updated.parquet')
df['binary_score'] = df['gpt4_evaluation'].apply(lambda x: 1 if x.lower().startswith('yes') else 0)

# Calculate and display accuracy
accuracy = df['binary_score'].mean()
print(f"\nAccuracy (percentage of correct answers): {accuracy:.2%}")

# Display the distribution of scores
print("\nDistribution of scores:")
print(df['binary_score'].value_counts())


Accuracy (percentage of correct answers): 61.50%

Distribution of scores:
binary_score
1    123
0     77
Name: count, dtype: int64


In [2]:
import os
import time
import pandas as pd
import yaml
from tqdm import tqdm
from openai import OpenAI
from HyDE import HyDEConfig, Promptor, HyDE

def get_gpt4_evaluation(ground_truth, model_response, openai_client):
    system_prompt = (
        "You are a helpful assistant. Please evaluate if the response matches the reference answer."
    )
    user_prompt = f"""Instructions
You will receive a ground truth answer (referred to as Answer) and a model-generated answer (referred to as Response).
Your task is to compare the two and determine whether they align.

Note: The ground truth answer may sometimes be embedded within the model-generated answer.
You need to carefully analyze and discern whether they align.

Your Output:
If the two answers align, respond with yes.
If they do not align, respond with no.
If you are very uncertain, respond with unclear.

Your response should first include yes, no, or unclear, followed by an explanation.

Example 1
Answer: Houston Rockets
Response: The basketball player who was drafted 18th overall in 2001 is Jason Collins, who was selected by the Houston Rockets.
Expected output: yes

Example 2
Answer: no
Response: Yes, both Variety and The Advocate are LGBT-interest magazines.
          The Advocate is explicitly identified as an American LGBT-interest magazine,
          while Variety, although primarily known for its coverage of the entertainment industry,
          also addresses topics relevant to the LGBT community.
Expected output: no

Input Data Format
Ground Truth Answer: {ground_truth}
Model Generated Answer: {model_response}

Expected Output
yes, no, or unclear
An explanation of your choice.

Output:
"""
    try:
        completion = openai_client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            max_tokens=500
        )
        evaluation_text = completion.choices[0].message.content.strip()
        return evaluation_text
    except Exception as e:
        print(f"Error calling GPT-4: {e}")
        return f"ERROR: {str(e)}"


def process_hotpotqa_questions(
    input_parquet: str,
    main_folder: str,
    output_parquet: str = None,
    openai_api_key: str = None
):
    openai_client = OpenAI(api_key=openai_api_key)
    df = pd.read_parquet(input_parquet)
    print(f"[INFO] Loaded HotpotQA data: {df.shape}")
    with open('Hconfig.yaml', 'r', encoding='utf-8') as f:
        base_config = yaml.safe_load(f)
    base_config['config']['main_folder'] = main_folder
    hyde_config = HyDEConfig(base_config)
    promptor = Promptor(hyde_config)
    hyde_obj = HyDE(hyde_config, promptor)

    for col in ["retrieval_document", "response", "processing_time", "gpt4_evaluation", "binary_score"]:
        if col not in df.columns:
            df[col] = None

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="HotpotQA-HyDE"):
        question = str(row['question'])
        start_time = time.time()
        retrieval_document = hyde_obj.e2e_search(question)
        end_time = time.time()
        processing_time = end_time - start_time

        if retrieval_document and isinstance(retrieval_document, list):
            best_hit = [retrieval_document[0]]
            response_data = hyde_obj.answer(best_hit, question)
        else:
            response_data = hyde_obj.answer([], question)

        df.at[idx, "retrieval_document"] = str(retrieval_document)
        df.at[idx, "response"] = response_data
        df.at[idx, "processing_time"] = processing_time

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="HotpotQA-GPT4-Eval"):
        ground_truth = str(row['answer'])
        model_resp = str(row['response'])
        gpt_eval = get_gpt4_evaluation(
            ground_truth=ground_truth,
            model_response=model_resp,
            openai_client=openai_client
        )
        df.at[idx, "gpt4_evaluation"] = gpt_eval

    def to_binary_score(eval_str: str):
        if not eval_str:
            return 0
        eval_str_lc = eval_str.strip().lower()
        if eval_str_lc.startswith("yes"):
            return 1
        else:
            return 0

    df["binary_score"] = df["gpt4_evaluation"].apply(to_binary_score)
    total_count = len(df)
    num_correct = df["binary_score"].sum()
    accuracy = num_correct / total_count if total_count > 0 else 0.0
    avg_processing_time = df["processing_time"].mean() if total_count > 0 else 0.0

    print(f"\n[STATS] Accuracy: {accuracy:.2%} ({num_correct}/{total_count})")
    print(f"[STATS] Average processing time: {avg_processing_time:.2f} seconds")
    if output_parquet:
        df.to_parquet(output_parquet, index=False)
        print(f"[INFO] Results saved to: {output_parquet}")

    return df


if __name__ == "__main__":
    input_parquet_path = "/Users/chengze/Desktop/hotpotqa_question_answer_type_200.parquet"
    main_folder_path = "/Users/chengze/Desktop/NaiveRAG_hotpotqa"
    output_parquet_path = "/Users/chengze/Desktop/NaiveRAG_hotpotqa_response_results_200.parquet"
    openai_api_key = ""

    df_result = process_hotpotqa_questions(
        input_parquet=input_parquet_path,
        main_folder=main_folder_path,
        output_parquet=output_parquet_path,
        openai_api_key=""
    )

    print("\n=== Final Data ===")
    print(df_result.head(10))


[INFO] Loaded HotpotQA data: (200, 3)


HotpotQA-GPT4-Eval: 100%|██████████| 200/200 [06:41<00:00,  2.01s/it]


[STATS] Accuracy: 66.00% (132/200)
[STATS] Average processing time: 2.92 seconds
[INFO] Results saved to: /Users/chengze/Desktop/NaiveRAG_hotpotqa_response_results_200.parquet

=== Final Data ===
                                               question                answer  \
1116  Out to Win is an American documentary film tha...       Houston Rockets   
1368  Are both Variety and The Advocate LGBT-interes...                    no   
422   Who is this American rapper, songwriter, recor...              Lil' Kim   
413   In which year was this American country music ...                  2000   
451   What is the nationality of the actor who starr...              Scottish   
861   Which tennis player is from the United States,...           James Arias   
1063  When did the company that produced "Six Gates ...               in 1986   
741   Interval starred Merle Oberon who fell in love...                Laredo   
1272  Which English Romantic poet wrote the sonnet t...  Percy Bysshe Shel

In [1]:
import os
import time
import asyncio
import pandas as pd
import tiktoken
import yaml
from tqdm import tqdm

import nest_asyncio
nest_asyncio.apply()

# ======== GraphRAG 相关导入 ========
from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.local_search.mixed_context import LocalSearchMixedContext
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

# ======== OpenAI GPT-4 相关 ========
import openai

# ======== HotpotQA Data + GPT-4 Evaluation ========

def get_gpt4_evaluation(ground_truth: str, model_response: str) -> str:
    """
    使用 GPT-4 对比 ground_truth 与 model_response
    并给出 "yes", "no", 或 "unclear" + 解释 的评估结果。
    """
    system_prompt = (
        "You are a helpful assistant. Please evaluate if the response matches the reference answer."
    )
    user_prompt = f"""Instructions
You will receive a ground truth answer (referred to as Answer) and a model-generated answer (referred to as Response).
Your task is to compare the two and determine whether they align.

Note: The ground truth answer may sometimes be embedded within the model-generated answer.
You need to carefully analyze and discern whether they align.

Your Output:
If the two answers align, respond with yes.
If they do not align, respond with no.
If you are very uncertain, respond with unclear.

Your response should first include yes, no, or unclear, followed by an explanation.

Example 1
Answer: Houston Rockets
Response: The basketball player who was drafted 18th overall in 2001 is Jason Collins, who was selected by the Houston Rockets.
Expected output: yes

Example 2
Answer: no
Response: Yes, both Variety and The Advocate are LGBT-interest magazines.
          The Advocate is explicitly identified as an American LGBT-interest magazine,
          while Variety, although primarily known for its coverage of the entertainment industry,
          also addresses topics relevant to the LGBT community.
Expected output: no

Input Data Format
Ground Truth Answer: {ground_truth}
Model Generated Answer: {model_response}

Expected Output
yes, no, or unclear
An explanation of your choice.

Output:
"""

    try:
        # 调用 GPT-4 ChatCompletion
        completion = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0,
            max_tokens=500
        )
        evaluation_text = completion.choices[0].message.content.strip()
        return evaluation_text
    except Exception as e:
        print(f"[ERROR calling GPT-4] {e}")
        return f"ERROR: {str(e)}"


async def process_hotpotqa_questions_graphrag(
    input_parquet: str,
    hotpotqa_folder: str,
    output_parquet: str = None,
    openai_api_key: str = None
):
    """
    使用 GraphRAG "Local Search" 对 hotpotqa_question_answer_type_200.parquet 的问题执行检索与回答，
    然后用 GPT-4 做评估。
    
    :param input_parquet:  HotpotQA 的问题 + 答案数据集路径 (e.g. /Users/chengze/Desktop/hotpotqa_question_answer_type_200.parquet)
    :param hotpotqa_folder: 包含 GraphRAG 索引产物 (entities, relationships, reports, text_units, etc.) 的文件夹路径
                            (e.g. /Users/chengze/Desktop/HotpotQA)
    :param output_parquet: 处理后的结果保存路径
    :param openai_api_key: 你的 OpenAI API Key
    """
    # ============= 1. 设置 OpenAI API Key =============
    if openai_api_key:
        openai.api_key = openai_api_key

    # ============= 2. 读取 HotpotQA 数据集 =============
    df = pd.read_parquet(input_parquet)
    print(f"[INFO] Loaded HotpotQA data: {df.shape}")

    # ============= 3. 加载 GraphRAG 索引产物 =============
    # 假设您在 /Users/chengze/Desktop/HotpotQA/output 下有：
    #   create_final_nodes.parquet, create_final_entities.parquet, create_final_relationships.parquet ...
    #   create_final_text_units.parquet, create_final_community_reports.parquet
    #   并且 lancedb/ 文件夹中有向量库
    OUTPUT_DIR = os.path.join(hotpotqa_folder, "output")
    LANCEDB_URI = os.path.join(OUTPUT_DIR, "lancedb")

    ENTITY_TABLE = "create_final_nodes"           # or create_final_entities
    RELATIONSHIP_TABLE = "create_final_relationships"
    COVARIATE_TABLE = "create_final_covariates"   # 如果您没有 covariates，可忽略
    COMMUNITY_REPORT_TABLE = "create_final_community_reports"
    TEXT_UNIT_TABLE = "create_final_text_units"

    entity_df = pd.read_parquet(os.path.join(OUTPUT_DIR, f"{ENTITY_TABLE}.parquet"))
    relationship_df = pd.read_parquet(os.path.join(OUTPUT_DIR, f"{RELATIONSHIP_TABLE}.parquet"))
    report_df = pd.read_parquet(os.path.join(OUTPUT_DIR, f"{COMMUNITY_REPORT_TABLE}.parquet"))
    text_unit_df = pd.read_parquet(os.path.join(OUTPUT_DIR, f"{TEXT_UNIT_TABLE}.parquet"))

    # 如果有 entity_embedding_df、covariate_df，也可加载
    # 如: entity_embedding_df = pd.read_parquet(...)
    #    covariate_df = pd.read_parquet(...)

    # 调用 GraphRAG 提供的帮助函数
    entities = read_indexer_entities(entity_df, entity_embedding_df=None, level=2)
    relationships = read_indexer_relationships(relationship_df)
    # 如果有 covariates
    #   covariates_df = pd.read_parquet(...)
    #   covariates = read_indexer_covariates(covariates_df)
    # 否则
    covariates = None

    reports = read_indexer_reports(report_df, entity_df, level=2)
    text_units = read_indexer_text_units(text_unit_df)

    # ============= 4. 连接向量数据库 (LanceDB) =============
    description_embedding_store = LanceDBVectorStore(collection_name="default-entity-description")
    description_embedding_store.connect(db_uri=LANCEDB_URI)

    # ============= 5. 初始化 ChatOpenAI, Embedding, LocalSearchMixedContext =============
    # 从环境变量或您自己的方式获取
    graphrag_api_key = os.environ.get("GRAPHRAG_API_KEY", openai_api_key or "")
    graphrag_llm_model = os.environ.get("GRAPHRAG_LLM_MODEL", "gpt-4o-mini")
    graphrag_embedding_model = os.environ.get("GRAPHRAG_EMBEDDING_MODEL", "text-embedding-3-small")

    llm = ChatOpenAI(
        api_key=graphrag_api_key,
        model=graphrag_llm_model,
        api_type=OpenaiApiType.OpenAI,  # 也可能是 AzureOpenAI
        max_retries=20,
    )
    token_encoder = tiktoken.get_encoding("cl100k_base")

    text_embedder = OpenAIEmbedding(
        api_key=graphrag_api_key,
        model=graphrag_embedding_model,
        api_type=OpenaiApiType.OpenAI,
        deployment_name=graphrag_embedding_model,
        max_retries=20,
    )

    # 构建 context builder
    context_builder = LocalSearchMixedContext(
        community_reports=reports,
        text_units=text_units,
        entities=entities,
        relationships=relationships,
        covariates=covariates,  # 如果没有 covariates，这里是 None
        entity_text_embeddings=description_embedding_store,
        embedding_vectorstore_key=EntityVectorStoreKey.ID,  # 如果 vector store 使用 entity 标题作为主键，可改为 TITLE
        text_embedder=text_embedder,
        token_encoder=token_encoder,
    )

    local_context_params = {
        "text_unit_prop": 0.5,
        "community_prop": 0.1,
        "conversation_history_max_turns": 5,
        "conversation_history_user_turns_only": True,
        "top_k_mapped_entities": 10,
        "top_k_relationships": 10,
        "include_entity_rank": True,
        "include_relationship_weight": True,
        "include_community_rank": False,
        "return_candidate_context": False,
        "embedding_vectorstore_key": EntityVectorStoreKey.ID,
        "max_tokens": 12000,
    }

    llm_params = {
        "max_tokens": 2000,
        "temperature": 0.0,
    }

    search_engine = LocalSearch(
        llm=llm,
        context_builder=context_builder,
        token_encoder=token_encoder,
        llm_params=llm_params,
        context_builder_params=local_context_params,
        response_type="local search response",  # 可根据需要调整
    )

    # ============= 6. 对 HotpotQA 中每条 "question" 执行检索 & 回答 + GPT-4 评价 =============
    # 先为结果 DataFrame 添加所需列
    for col in ["retrieval_document", "response", "processing_time", "gpt4_evaluation", "binary_score"]:
        if col not in df.columns:
            df[col] = None

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="GraphRAG-LocalSearch"):
        question = str(row["question"])
        start_time = time.time()

        # 如果 GraphRAG 的 search_engine 只提供异步 asearch
        # 这里需 await 调用
        result = await search_engine.asearch(question)

        end_time = time.time()
        processing_time = end_time - start_time

        # result 的结构中包含 result.response 以及 result.context_data
        # 视 GraphRAG 不同版本而异
        retrieval_document = result.context_data.get("entities", [])
        # 也可以根据 context_data["sources"] / context_data["relationships"] 等做自定义拼接

        response_data = result.response

        # 回填
        df.at[idx, "retrieval_document"] = str(retrieval_document)
        df.at[idx, "response"] = response_data
        df.at[idx, "processing_time"] = processing_time

    # ============= 7. GPT-4 评价 =============
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="GPT4-Evaluation"):
        ground_truth = str(row["answer"])
        model_resp = str(row["response"])
        gpt_eval = get_gpt4_evaluation(ground_truth, model_resp)
        df.at[idx, "gpt4_evaluation"] = gpt_eval

    # ============= 8. 将 yes/no/unclear → 二元分数 =============
    def to_binary_score(eval_str: str):
        if not eval_str:
            return 0
        eval_str_lc = eval_str.strip().lower()
        if eval_str_lc.startswith("yes"):
            return 1
        else:
            return 0

    df["binary_score"] = df["gpt4_evaluation"].apply(to_binary_score)

    # ============= 9. 统计准确率 & 平均处理时长 =============
    total_count = len(df)
    num_correct = df["binary_score"].sum()
    accuracy = num_correct / total_count if total_count > 0 else 0.0
    avg_processing_time = df["processing_time"].mean() if total_count > 0 else 0.0

    print(f"\n[STATS] Accuracy: {accuracy:.2%} ({num_correct}/{total_count})")
    print(f"[STATS] Average processing time: {avg_processing_time:.2f} seconds")

    # ============= 10. 保存到新的 parquet 文件 =============
    if output_parquet:
        df.to_parquet(output_parquet, index=False)
        print(f"[INFO] Results saved to: {output_parquet}")

    return df


def main():
    # 假设您 HotpotQA folder 为 "/Users/chengze/Desktop/HotpotQA"
    # 其中包含 output/ 文件夹存放 Graphrag 索引产物
    hotpotqa_folder = "/Users/chengze/Desktop/HotpotQA"

    # 输入数据集（问题 + 答案）
    input_parquet = "/Users/chengze/Desktop/hotpotqa_question_answer_type_200.parquet"

    # 输出结果文件
    output_parquet = "/Users/chengze/Desktop/GraphRAG_hotpotqa_response_results_200.parquet"

    # 设置您的 OpenAI Key
    openai_api_key = ""

    # 以异步方式运行
    asyncio.run(process_hotpotqa_questions_graphrag(
        input_parquet=input_parquet,
        hotpotqa_folder=hotpotqa_folder,
        output_parquet=output_parquet,
        openai_api_key=openai_api_key
    ))

if __name__ == "__main__":
    main()


[INFO] Loaded HotpotQA data: (200, 3)


TypeError: read_indexer_entities() got an unexpected keyword argument 'entity_embedding_df'

In [3]:
import graphrag.query.indexer_adapters as ia
help(ia.read_indexer_entities)


Help on function read_indexer_entities in module graphrag.query.indexer_adapters:

read_indexer_entities(final_nodes: pandas.core.frame.DataFrame, final_entities: pandas.core.frame.DataFrame, community_level: int | None) -> list[graphrag.model.entity.Entity]
    Read in the Entities from the raw indexing outputs.

